In [1]:
%matplotlib inline

import os
import shutil
import time

from IPython.display import Image
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import DataLoader
from densenet_modified import densenet201

In [2]:
# Trainer parameters
print_freq_epochs = 100
use_cuda = True

# Dataset Parameters
batch_size = 64
load_size = 128
fine_size = 128
c = 3
data_mean = np.asarray([0.45834960097,0.44674252445,0.41352266842])

# Training parameters
# architecture = 'resnet34'
architecture = 'dense'
lr = 0.1  # Can be large for resnet or if using batch norm
momentum = 0.90
weight_decay = 1e-4
num_epochs = 100

In [3]:
def construct_dataloader_disk():
    # Construct DataLoader
    opt_data_train = {
        #'data_h5': 'miniplaces_128_train.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/train.txt', # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': True
        }
    opt_data_val = {
        #'data_h5': 'miniplaces_128_val.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/val.txt',   # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
        }

    loader_train = DataLoader.DataLoaderDisk(**opt_data_train)
    loader_val = DataLoader.DataLoaderDisk(**opt_data_val)
    
    return (loader_train, loader_val)

def construct_dataloader_h5():
    # Construct DataLoader from an h5 file
    opt_data_train = {
        'data_h5': 'miniplaces_128_train.h5',
        #'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        #'data_list': '../../data/train.txt', # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': True
        }
    opt_data_val = {
        'data_h5': 'miniplaces_128_val.h5',
        #'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        #'data_list': '../../data/val.txt',   # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
        }

    loader_train = DataLoader.DataLoaderH5(**opt_data_train)
    loader_val = DataLoader.DataLoaderH5(**opt_data_val)
    
    return (loader_train, loader_val)

In [4]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def adjust_learning_rate(lr, optimizer, epoch):
    """Calculates a learning rate of the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def save_checkpoint(model, state, is_best, epoch):
    torch.save(state, "models/"+"checkpoint_dense201.tar")
    if is_best:
        torch.save(model, "models/"+"best_dense201.pt")

In [5]:
# train and validate methods adapted from https://github.com/pytorch/examples/blob/master/imagenet/main.py

def train(train_loader, model, criterion, optimizer, epoch, text_file):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i in range(int(train_loader.size()/batch_size)):
        input, target = train_loader.next_batch(batch_size)
        target = target.long()
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)
        target_var = target_var.long()
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        text_file.write(str(epoch)+str(",")+str(i)+str(",")+str(batch_time.val)+str(",")+str(data_time.val)+str(",")+str(losses.val)+str(",")+str(top1.val)+str(",")+str(top5.val)+"\n")
        
        if i % print_freq_epochs == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, train_loader.size()/batch_size, batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))


def validate(val_loader, model, criterion, text_file):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i in range(int(val_loader.size()/batch_size)):
        input, target = val_loader.next_batch(batch_size)
        target = target.long()
        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        target_var = target_var.long()

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        text_file.write(str("val, ")+str(i)+str(",")+str(batch_time.val)+str(",")+str(losses.val)+str(",")+str(top1.val)+str(",")+str(top5.val)+"\n")
        
        if i % print_freq_epochs == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, val_loader.size()/batch_size, batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))


    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top5.avg

In [10]:
if architecture[:5]!='dense':
    model = models.__dict__[architecture]()
else:
    model = densenet201()
    
criterion = nn.CrossEntropyLoss()

if use_cuda:
    criterion = criterion.cuda()
    model = model.cuda()

In [11]:
# model.avgpool = nn.AdaptiveAvgPool2d(1) # to be used for ResNets 

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

In [13]:
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=weight_decay)

In [14]:
# train_loader, val_loader = construct_dataloader_disk()
train_loader, val_loader = construct_dataloader_h5()

# Images found: 100000
# Images found: 10000


In [9]:
# model = torch.load('resnet504-3.0.ph')

In [15]:
best_prec5 = 0.

In [20]:
for epoch in range(1,num_epochs):
    lr = adjust_learning_rate(lr, optimizer, epoch)

    text_file = open("epoch_output_densenet201.txt", "w")
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, text_file)

    # evaluate on validation set
    prec5 = validate(val_loader, model, criterion, text_file)
    text_file.close()
    
    # remember best prec@1 and save checkpoint
    is_best = prec5 > best_prec5
    best_prec5 = max(prec5, best_prec5)
    save_checkpoint(model, {
        'epoch': epoch + 1,
        'arch': architecture,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec5,
        'optimizer' : optimizer.state_dict(),
    }, is_best, epoch)

Epoch: [1][0/1562.5]	Time 0.445 (0.445)	Data 0.034 (0.034)	Loss 6.8916 (6.8916)	Prec@1 0.000 (0.000)	Prec@5 1.562 (1.562)


KeyboardInterrupt: 

In [19]:
dummy_text_file = open("epoch_outputXX.txt", "w")
prec1 = validate(val_loader, model, criterion, dummy_text_file)

Test: [0/156.25]	Time 0.119 (0.119)	Loss 6.9066 (6.9066)	Prec@1 1.562 (1.562)	Prec@5 3.125 (3.125)
Test: [100/156.25]	Time 0.102 (0.102)	Loss 6.9088 (6.9075)	Prec@1 0.000 (1.006)	Prec@5 0.000 (1.083)
 * Prec@1 0.891 Prec@5 0.992


In [63]:
torch.save(model,'densenet_ep23_asymptotic.pt')

In [65]:
best_prec5

80.66907051282051

In [22]:
dense_model = torch.load('models/best_dense.pt')
dense_model2 = torch.load('models/best_dense201.pt')
resnet_model = torch.load('models/best_resnet34.pt')

In [17]:
def validate_ensamble(val_loader, model1, model2, model3, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i in range(int(val_loader.size()/batch_size)):
        input, target = val_loader.next_batch(batch_size)
        target = target.long()
        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        target_var = target_var.long()

        # compute output
        output1 = model1(input_var)
        output2 = model2(input_var)
        output3 = model3(input_var)
        output = output3+output1+output2
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
                
        if i % print_freq_epochs == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, val_loader.size()/batch_size, batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))


    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top5.avg

In [18]:
validate_ensamble(val_loader, dense_model, dense_model, dense_model2, criterion)

Test: [0/156.25]	Time 0.872 (0.872)	Loss 2.8430 (2.8430)	Prec@1 57.812 (57.812)	Prec@5 85.938 (85.938)
Test: [100/156.25]	Time 0.191 (0.194)	Loss 3.3035 (3.6776)	Prec@1 59.375 (54.394)	Prec@5 85.938 (82.147)
 * Prec@1 54.137 Prec@5 82.302


82.3016826923077